# Metrics, validation strategies and baselines

В данном jupyter notebook рассматриваются примеры того, какие схемы валидации и метрики используются в рекомендательных системах.
Также построим простые модели (бейзлайны) на данных МТС Библиотеки. 

* [Preprocessing](#preprocessing)
* [General remarks](#general-remarks)
* [Metrics](#metrics)
    * [Regression](#regression)
    * [Classification](#classification)
    * [Ranking](#ranking)
* [Validation strategies](#validation)
* [Baselines](#baselines)

In [1]:
import os
import numpy as np 
import pandas as pd 
from itertools import islice, cycle
from more_itertools import pairwise



<a id="preprocessing"></a>
# Preprocessing

Загрузим наши данные, теперь уже с фичами, и применим знания из [pandas-scipy-for-recsys](https://www.kaggle.com/sharthz23/pandas-scipy-for-recsys)

In [2]:
df = pd.read_pickle('./interactions_preprocessed.pickle')
df_users = pd.read_pickle('./users_preprocessed.pickle')
df_items = pd.read_pickle('./items_preprocessed.pickle')

In [3]:
test_dates = df['start_date'].unique()[-7:]



In [4]:
test_dates = list(pairwise(test_dates))
test_dates

[(numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (numpy.datetime64('2019-12-26T00:00:00.000000000'),
  numpy.datetime64('2019-12-27T00:00:00.000000000')),
 (numpy.datetime64('2019-12-27T00:00:00.000000000'),
  numpy.datetime64('2019-12-28T00:00:00.000000000')),
 (numpy.datetime64('2019-12-28T00:00:00.000000000'),
  numpy.datetime64('2019-12-29T00:00:00.000000000')),
 (numpy.datetime64('2019-12-29T00:00:00.000000000'),
  numpy.datetime64('2019-12-30T00:00:00.000000000')),
 (numpy.datetime64('2019-12-30T00:00:00.000000000'),
  numpy.datetime64('2019-12-31T00:00:00.000000000'))]

In [5]:
split_dates = test_dates[0]

In [6]:
train = df[df['start_date'] < split_dates[0]]

In [7]:
test = df[(df['start_date'] >= split_dates[0]) & (df['start_date'] < split_dates[1])]

In [8]:
test = test[(test['rating'] >= 4) | (test['rating'].isnull())]

In [9]:
split_dates, train.shape, test.shape


((numpy.datetime64('2019-12-25T00:00:00.000000000'),
  numpy.datetime64('2019-12-26T00:00:00.000000000')),
 (1517994, 5),
 (2114, 5))

<a id="baselines"></a>
# Baselines

Самым популярным бейзлайном является просто построение популярного :)
Гиперпараметром такой модели может быть например окно, за которое мы считаем популярное.

Модель можно расширять засчет учета фичей, чтобы считать популярное в рамках каких-то групп.

Но на самом деле бейзлайны в первую очередь зависят от типа данных. В некоторых случаях это могут быть простые модели (или даже бизнес-правила), которые просто учитывают контекст задачи.

In [10]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []

    def fit(self, df, ):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values

    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

In [11]:
pop_model = PopularRecommender(days=77, dt_column='start_date')

In [12]:
vars(pop_model)

{'max_K': 100,
 'days': 77,
 'item_column': 'item_id',
 'dt_column': 'start_date',
 'recommendations': []}

In [15]:
pop_model.fit(train)

In [16]:
vars(pop_model)

{'max_K': 100,
 'days': 77,
 'item_column': 'item_id',
 'dt_column': 'start_date',
 'recommendations': array([109201, 230067, 235407,  35265, 155266, 237760, 291806, 151190,
        270415, 285394, 218025, 115190, 147734, 282647, 208935, 271846,
         74650, 281005, 168037, 135032, 168900, 219099, 288531,  75579,
        226196,  96052,   9197, 169853, 312940, 103077,  99616,  63978,
          6136, 311394,  86588,  43622, 295007, 110811, 104010, 176898,
        210979,  48510, 107522,  55913,  67643, 181955,  30772, 141961,
        233762, 126581, 247959,  93249, 238200,  39878, 276498, 238856,
        303990,  86123,   5001,   8875,  31454, 241026,  28889, 209326,
        265984,  26963, 231923, 211217, 243711, 121687,  90674, 108759,
        232317, 163618, 251856,  11424, 233644,  58803, 240571,  49352,
        231257,  12301, 297013, 145016,  65912,  99357, 181857, 226128,
         81827,  35226, 121544, 206185, 312792,  90225,  34363, 223806,
        222102, 250772,  25173, 22

In [17]:
pop_model.recommendations

array([109201, 230067, 235407,  35265, 155266, 237760, 291806, 151190,
       270415, 285394, 218025, 115190, 147734, 282647, 208935, 271846,
        74650, 281005, 168037, 135032, 168900, 219099, 288531,  75579,
       226196,  96052,   9197, 169853, 312940, 103077,  99616,  63978,
         6136, 311394,  86588,  43622, 295007, 110811, 104010, 176898,
       210979,  48510, 107522,  55913,  67643, 181955,  30772, 141961,
       233762, 126581, 247959,  93249, 238200,  39878, 276498, 238856,
       303990,  86123,   5001,   8875,  31454, 241026,  28889, 209326,
       265984,  26963, 231923, 211217, 243711, 121687,  90674, 108759,
       232317, 163618, 251856,  11424, 233644,  58803, 240571,  49352,
       231257,  12301, 297013, 145016,  65912,  99357, 181857, 226128,
        81827,  35226, 121544, 206185, 312792,  90225,  34363, 223806,
       222102, 250772,  25173, 229749], dtype=int64)

In [18]:
pop_model.days

77

In [21]:
item_titles = dict(zip(df_items.id,df_items.title))

In [24]:
item_titles[128115]

'Ворон-челобитчик'

In [25]:
print(item_titles.get(128115))

Ворон-челобитчик


In [55]:
top10_recs = pop_model.recommend()
top10_recs

array([109201, 230067, 235407,  35265, 155266, 237760, 291806, 151190,
       270415, 285394], dtype=int64)

In [57]:
map(item_titles.get, top10_recs)

In [56]:
list(map(item_titles.get, top10_recs))

['Яблоки из сада Шлицбутера',
 'Кавказский пленник',
 'Пикник на обочине',
 'Записки юного врача',
 'О любви',
 'Русские народные сказки',
 'Женская война',
 'История государства Российского. Том 2. От Великого князя Святополка до Великого князя Мстислава Изяславовича',
 'Черный человек',
 'Хитрость']

In [30]:
# create datframe with user_id fo recs
recs = pd.DataFrame({'user_id': test['user_id'].unique()})


In [32]:
recs.head(2)

,user_id
0,38753
1,101642


In [33]:

top_N = 10
recs['item_id'] = pop_model.recommend(recs['user_id'], N=top_N)
recs.head()

,user_id,item_id
0,38753,"[109201, 230067, 235407, 35265, 155266, 237760..."
1,101642,"[109201, 230067, 235407, 35265, 155266, 237760..."
2,13548,"[109201, 230067, 235407, 35265, 155266, 237760..."
3,130425,"[109201, 230067, 235407, 35265, 155266, 237760..."
4,93986,"[109201, 230067, 235407, 35265, 155266, 237760..."
